In [22]:
import json
long_terms = ['<>', 'id', 'end', 'begin', '>=', '<=']
simple_terms = ['(', '/', ')', '%', '^', '-', '*', ';', '+'] + ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
short_terms = ['=', '>', '<']

In [23]:
def expression(i):
    node = []
    i, arif_expr_node = arif_expression(i)
    node.append({"arif_expression":arif_expr_node})
    i, relationship_sign_node = relationship_sign(i)
    node.append({"relationship_sign":relationship_sign_node})
    i, arif_expr_node = arif_expression(i)
    node.append({"arif_expression":arif_expr_node})
    return i, node


def arif_expression(i):
    node = []
    try:
        i, add_type_node = add_type_expr(i)
        node.append({"add_type_expr":add_type_node})
    except:
        pass
    i, term_node = term(i)
    node.append({"term":term_node})
    try:
        i, arif_expression_1_node = arif_expression_1(i)
        node.append({"arif_expression_1":arif_expression_1_node})
    except:
        pass
    return i, node


def term(i):
    node = []
    i, mult_node = mult(i)
    node.append({"mult":mult_node})
    try:
        i, term_1_node = term_1(i)
        node.append({"term'":term_1_node})
    except:
        pass
    return i, node


def mult(i):
    node = []
    i, first_expr_node = first_expr(i)
    node.append({"first_expr":first_expr_node})
    try:
        i, mult_1_node = mult_1(i)
        node.append({"mult_1":mult_1_node})
    except:
        pass
    return i, node


def first_expr(i):
    node = []
    if tokenized[i] == "(":
        node.append("(")
        i = i+1
        i,arif_expression_node = arif_expression(i)
        node.append({"arif_expression":arif_expression_node})
        if tokenized[i] == ")":
            i = i+1
            node.append(")")
        else:
            raise ValueError(i)
        return i, node
    try:
        i, number_node = number(i)
        node.append({"number":number_node})
    except:
        i, id_node = identifier(i)
        node.append({"identifier":id_node})
    return i, node


def add_type_expr(i):
    if tokenized[i] in ("+", "-"):
        i = i+1
        return i, [tokenized[i-1]]
    else:
        raise ValueError(i)


def mult_type_expr(i):
    if tokenized[i] in ("*", "/", "%"):
        i = i+1
        return i, [tokenized[i-1]]
    else:
        raise ValueError(i)
    

def relationship_sign(i):
    if tokenized[i] in ("<", "<=", "=", ">=", ">", "<>"):
        i = i+1
        return i, [tokenized[i-1]]
    else:
        raise ValueError(i)


def identifier(i):
    if tokenized[i] == "id":
        i = i+1
        return i, [tokenized[i-1]]
    else:
        raise ValueError(i)


def number(i):
    if tokenized[i] in ("0", "1", "2", "3", "4", "5", "6", "7", "8", "9"):
        i = i+1
        return i, [tokenized[i-1]]
    else:
        raise ValueError(i)


def term_1(i):
    node = []
    i, mult_expr_node = mult_type_expr(i)
    node.append({"mult_type_expr":mult_expr_node})
    i, mult_node = mult(i)
    node.append({"mult":mult_node})
    try:
        i, term_1_node = term_1(i)
        node.append({"temr'":term_1_node})
    except:
        pass
    return i, node


def mult_1(i):
    node = []
    if tokenized[i] == "^":
        i+= 1
        node.append("^")
    else:
        raise ValueError(i)
    i, first_expr_node = first_expr(i)
    node.append({"first_expr":first_expr_node})
    try:
        i, mult_1_node = mult_1(i)
        node.append({"mult'":mult_1_node})
    except:
        pass
    return i, node


def arif_expression_1(i):
    node = []
    i, add_expr_node = add_type_expr(i)
    node.append({"add_expr":add_expr_node})
    i, term_node = term(i)
    node.append({"term":term_node})
    try:
        i, are1_node = arif_expression_1(i)
        node.append({"arif_expression_1":are1_node})
    except:
        pass
    return i, node

In [24]:


def tokenize(in_str):
    tokenized = []
    i = 0
    while i<len(in_str):
        if in_str[i] in simple_terms:
            tokenized.append(in_str[i])
            i = i+1
            continue
        for t in long_terms:
            flag = False
            if in_str[i:].startswith(t):
                tokenized.append(t)
                i+=len(t)
                flag = True
                break
        if flag:
            continue

        if in_str[i] in short_terms:
            tokenized.append(in_str[i])
            i = i+1
            continue
        
        i = i+1
    return tokenized

In [32]:
# tokenized = tokenize("3*(1+2*3+4)>5^6")
tokenized = tokenize("3*(1+2*3+4)>5^6+1/2")
# tokenized = tokenize("3+7*2>0")
tokenized

['3',
 '*',
 '(',
 '1',
 '+',
 '2',
 '*',
 '3',
 '+',
 '4',
 ')',
 '>',
 '5',
 '^',
 '6',
 '+',
 '1',
 '/',
 '2']

In [33]:
checksum, tree = expression(0)

In [34]:
def make_polish(node):
    nums_to_skip = []
    # print(node)
    if type(node)==str and node not in "()":
        res.append(node)
        return
    if type(node)==list:
        
        for i, a in enumerate(node):
            if a == "^":
                nums_to_skip.append(i)
            if type(a) == str:
                continue
            if list(a.keys())[0] in ("mult_type_expr", "add_expr", "relationship_sign"):
                nums_to_skip.append(i)
        for i, a in enumerate(node):
            if i not in nums_to_skip:
                make_polish(a)
    if type(node)==dict:
        for _, i in node.items():
            make_polish(i)
    for i in nums_to_skip:
        make_polish(node[i])
    return


In [35]:
json.dumps(tree)

'[{"arif_expression": [{"term": [{"mult": [{"first_expr": [{"number": ["3"]}]}]}, {"term\'": [{"mult_type_expr": ["*"]}, {"mult": [{"first_expr": ["(", {"arif_expression": [{"term": [{"mult": [{"first_expr": [{"number": ["1"]}]}]}]}, {"arif_expression_1": [{"add_expr": ["+"]}, {"term": [{"mult": [{"first_expr": [{"number": ["2"]}]}]}, {"term\'": [{"mult_type_expr": ["*"]}, {"mult": [{"first_expr": [{"number": ["3"]}]}]}]}]}, {"arif_expression_1": [{"add_expr": ["+"]}, {"term": [{"mult": [{"first_expr": [{"number": ["4"]}]}]}]}]}]}]}, ")"]}]}]}]}]}, {"relationship_sign": [">"]}, {"arif_expression": [{"term": [{"mult": [{"first_expr": [{"number": ["5"]}]}, {"mult_1": ["^", {"first_expr": [{"number": ["6"]}]}]}]}]}, {"arif_expression_1": [{"add_expr": ["+"]}, {"term": [{"mult": [{"first_expr": [{"number": ["1"]}]}]}, {"term\'": [{"mult_type_expr": ["/"]}, {"mult": [{"first_expr": [{"number": ["2"]}]}]}]}]}]}]}]'

In [36]:
res = []
make_polish(tree)

In [37]:
"".join(res)

'3123*4++*56^12/+>'

In [38]:
"".join(tokenized)

'3*(1+2*3+4)>5^6+1/2'